In [1]:
pip install pyamaze

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Import necessary modules.
from collections import deque

def BFS(m, start=None):
    # Define the start cell, if not provided use the bottom-right cell.
    if start is None:
        start = (m.rows, m.cols)

    # Initialize the frontier as a deque (double-ended queue), start with the start cell.
    frontier = deque()
    frontier.append(start)

    # Dictionary to store the path taken by BFS.
    bfsPath = {}

    # List to keep track of the cells that have already been explored.
    explored = [start]

    # List to store the order of cells explored.
    bfsSearch = []

    # Process while there are cells in the frontier.
    while len(frontier) > 0:
        # Dequeue a cell from the frontier.
        currCell = frontier.popleft()

        # If the current cell is the goal, then stop processing.
        if currCell == m._goal:
            break

        # Check each direction from the current cell.
        for d in 'ESNW':
            # If the current direction is passable (no wall)...
            if m.maze_map[currCell][d] == True:
                # Determine the coordinates of the child cell based on the direction.
                if d == 'E':
                    childCell = (currCell[0], currCell[1] + 1)
                elif d == 'W':
                    childCell = (currCell[0], currCell[1] - 1)
                elif d == 'S':
                    childCell = (currCell[0] + 1, currCell[1])
                elif d == 'N':
                    childCell = (currCell[0] - 1, currCell[1])

                # If the child cell has already been explored, skip this iteration.
                if childCell in explored:
                    continue

                # Add the child cell to the frontier and mark it as explored.
                frontier.append(childCell)
                explored.append(childCell)

                # Add the child cell and its parent to the BFS path. Also, update the search order list.
                bfsPath[childCell] = currCell
                bfsSearch.append(childCell)

    # Trace the path from the goal to the start.
    fwdPath = {}
    cell = m._goal
    while cell != (m.rows, m.cols):
        fwdPath[bfsPath[cell]] = cell
        cell = bfsPath[cell]

    # Return the search order, path, and forward path.
    return bfsSearch, bfsPath, fwdPath

In [3]:
from pyamaze import maze, agent, COLOR, textLabel
from queue import PriorityQueue

# Define the heuristic function.
def h(cell1, cell2):
    # Calculate Manhattan distance between two cells.
    x1, y1 = cell1
    x2, y2 = cell2
    return abs(x1-x2)+abs(y1-y2)

# A* algorithm function.
def aStar(m, start=None):
    # If no start cell is provided, choose the bottom right cell as the start cell.
    if start is None:    
        start=(m.rows,m.cols)
    # PriorityQueue of open cells.
    open=PriorityQueue()
    open.put((h(start, m._goal), h(start, m._goal), start))

    # Dictionary to store the path taken by A*.
    aPath={}

    # Initialize g_score (cost from start to cell) and f_score (g_score + heuristic) for each cell.
    g_score = {row: float("inf") for row in m.grid}
    g_score[start] = 0
    f_score = {row: float("inf") for row in m.grid}
    f_score[start]=h(start, m._goal)
    # List to store the order of cells explored.
    searchPath=[start]
    # Process while there are cells in the open PriorityQueue.
    while not open.empty():
        # Dequeue a cell from the PriorityQueue.
        currCell=open.get()[2]
        # Add the current cell to the searchPath.
        searchPath.append(currCell)
        # If the current cell is the goal, then stop processing.
        if currCell==m._goal:
            break
        # Check each direction from the current cell.
        for d in 'ESNW':
            # If the current direction is passable (no wall)...
            if m.maze_map[currCell][d]==True:
                # Calculate the coordinates of the child cell based on the direction.
                if d=='E':
                    childCell=(currCell[0],currCell[1]+1)
                if d=='W':
                    childCell=(currCell[0],currCell[1]-1)
                if d=='S':
                    childCell=(currCell[0]+1,currCell[1])
                if d=='N':
                    childCell=(currCell[0]-1,currCell[1])
                
                # Calculate tentative g_score and f_score for the child cell.
                temp_g_score = g_score[currCell] + 1
                temp_f_score = temp_g_score + h(childCell, m._goal)

                # If the new f_score is lower, update the path and scores for the child cell.
                if temp_f_score < f_score[childCell]:
                    aPath[childCell]=currCell
                    g_score[childCell] = temp_g_score
                    f_score[childCell] = temp_f_score
                    # Add the child cell to the PriorityQueue with the new f_score.
                    open.put((f_score[childCell], h(childCell,m._goal), childCell))
                    
    # Trace the path from the goal to the start.
    fwdPath={}
    cell=m._goal
    while cell!=start:
        fwdPath[aPath[cell]]=cell
        cell=aPath[cell]
    # Return the search order, path, and forward path.
    return searchPath,aPath,fwdPath

In [4]:
# Import the necessary modules and functions
from pyamaze import maze, agent, COLOR, textLabel  # Pyamaze library for creating and visualizing mazes
from timeit import timeit  # Function for measuring execution time

# Create a maze instance
m = maze()

# Load the maze layout from a CSV file
m.CreateMaze(loadMaze='mazemain.csv')

# Run the BFS algorithm on the maze and get the paths
bSearch, bfsPath, fwdBFSPath = BFS(m)

# Display the lengths of the paths found by BFS on the maze
textLabel(m, 'BFS Path Length', len(fwdBFSPath)+1)
textLabel(m, 'BFS Search Length', len(bSearch)+1)

# Measure the execution time of BFS
t1 = timeit('BFS(m)', number=500, globals=globals())  # Time for BFS

# Display the execution time on the maze
textLabel(m, 'BFS Time', t1)

# Create agents for visualizing the paths found by BFS
a = agent(m, footprints=True, color=COLOR.blue, filled=True)
b = agent(m,1,1, footprints=True, color=COLOR.yellow, filled=True, goal=(m.rows, m.cols))
c = agent(m, footprints=True, color=COLOR.red)

# Have the agents trace the paths found by BFS
m.tracePath({a: bSearch}, delay=40)
m.tracePath({b: bfsPath}, delay=80)
m.tracePath({c: fwdBFSPath}, delay=80)

# Run the A* algorithm on the maze and get the paths
aSearch, aStarPath, fwdAStarPath = aStar(m)

# Display the lengths of the paths found by A* on the maze
textLabel(m, 'A* Path Length', len(fwdAStarPath)+1)
textLabel(m, 'A* Search Length', len(aSearch)+1)

# Measure the execution time of A*
t2 = timeit('aStar(m)', number=500, globals=globals())  # Time for A*

# Display the execution time on the maze
textLabel(m, 'A* Time', t2)

# Create agents for visualizing the paths found by A*
a = agent(m, footprints=True, color=COLOR.green, filled=True)
b = agent(m,1,1, footprints=True, color=COLOR.yellow, filled=True, goal=(m.rows, m.cols))
c = agent(m, footprints=True, color=COLOR.red)

# Have the agents trace the paths found by A*
m.tracePath({a: aSearch}, delay=40)
m.tracePath({b: aStarPath}, delay=80)
m.tracePath({c: fwdAStarPath}, delay=80)

# Start the visualization
m.run()